In [37]:
# encoding: utf-8
# 17.22 word transformer
from string import ascii_lowercase
from collections import defaultdict, deque

# Solution 1: DFS
# Helper function that recursively finds the path from start to target
def DFS_words(visited, start, target, dictionary):
    """
    Given a dictionary, build a path from the start word to end word. Each step only change on character
    """
    if start ==  target:
        return [target]
    
    # Find all words that are one character away from start
    # recursively call DFS_words() from that new word
    # Use the visited set to avoid visiting repeatly and use dictionary to avoid visiting impossible words
    else:
        visited.add(start)
        
        for i in range(len(start)):
            for c in ascii_lowercase:
                new_start = list(start)
                new_start[i] = c
                new_start = "".join(new_start)
                if (new_start not in visited) and (new_start in dictionary):
                    path = DFS_words(visited, new_start, target, dictionary)
                    
                    if path != None:
                        path.append(start)
                        return path
    return None

# Main function for DFS
def word_transform_dfs(start, target, dictionary):
    visited = set([])
    path = DFS_words(visited, start, target, dictionary)
    
    if path != None:
        path.reverse()
        return path
    else:
        return None
    

# Solution 2: BFS from both start and targets
# By doing BFS from two sides, it can reduce the average search distance by half, which means sqrt the time complexity

# Build a network structure of how a word related to a neibourghing word
class Word_Node:
    def __init__(self, word):
        self.w = word

def build_map(dictionary):
    """
    For each word in the dictionary, build a node. Then build the relationships between the node
    """
    # Build a mapping between words to Node
    words = {}
    for word in dictionary:
        words[word] = Word_Node(word)
    
    # Build a mapping between Nodes
    map = defaultdict(list)
    for word, node in words.items():
        # For each possible one-character variation
        for i in range(len(w)):
            for char in ascii_lowercase:
                variation = list(start)
                variation[i] = c
                variation = "".join(variation)
                if variation != word and variation in words: # if exist
                    map[node].append(words[variation]) # add to the node-to-node mapping
    
    return map, words

class BFS_Data():
    
    def __init__(self):
        """
        The object to save the essential data structure to do BFS
        """
        self.frontier = deque() # Pythonic queue
        self.visited = set([]) # record visited nodes
        self.previous_visit = {} # record which node leads to which
    
    def gather_path(self, node, start_to_end =True):
        """
        Find a path to the given node
        """
        path = []
        while len(self.previous_visit[node]) != 0:
            path.append(node) # earlier visited node is closer to the end of list
            node = self.previous_visit[node]
       
        if start_to_end == True: # If we want a path from start to end
            path.reverse() # reverse it
        
        return path

def search_one_layer(primary, secondary, map, words):
    """
    BFS from two sides for one round each, if collide, return the collision point
    """
    # Check if the primary and secondary are both done
    if len(primary.frontier) == 0 and len(secondary.frontier) == 0:
        raise Exception("No more BFS") # kind of a flag
    
    # There is stuff to BFS
    # Primary do a round of BFS first
    size_frontier = len(primary.frontier)
    for _ in range(size_frontier):
        node = primary.frontier.popleft()
        if node in secondary.frontier: # two BFS met!
            return node
        else: # not met
            primary.visited.add(node) # visited
            for neighbour in map[node]:
                if neighbour not in primary.visited:
                    primary.frontier.append(neighbour)
                    primary.previous_visit[neighbour] = node # this neighbour is reachable from node
    
    # Secondary do a round of BFS, the same as primary
    size_frontier = len(secondary.frontier)
    for _ in range(size_frontier):
        node = secondary.frontier.popleft()
        if node in primary.frontier: # two BFS met!
            return node
        else: # not met
            secondary.visited.add(node) # visited
            for neighbour in map[node]:
                if neighbour not in secondary.visited:
                    secondary.frontier.append(neighbour)
                    secondary.previous_visit[neighbour] = node # this neighbour is reachable from node
    
    return None # if no collision, return None

# Main function for BFS
def word_transform(start, target, dictionary):
    
    #  Check if start and target in the dictionary
    if (start not in dictionary) or (target not in dictionary):
        raise Exception("Start ot target word not in the dictionary!")
    
    # Build the network
    map, words = build_map(dictionary)
    
    # Initialize two objects for BFS
    path = []
    primary, secondary = BFS_Data()
    primary.frontier.append(words[start])
    secondary.frontier.append(words[target])
    
    # Keep doing two-ended BFS until collision or raise exception because there's no path from start to end
    while True:
        try:
            res = search_one_layer(primary, secondary, map, words)
            if res != None:
                path_start_to_collision = primary.gather_path(res, True)
                path_collision_to_target = secondary.gather_path(res)
                path_target_to_collision.pop(0) # remove the collision node to avoid repeat
                path = path_start_to_collision + path_collision_to_target
                return path
        
        except: # Can't find the path
            raise Exception("Start ot target word not in the dictionary!")

In [32]:
# Test
dictionary = set(["damp", "like", "lamp", "limp", "lime", "haha", "hehe"])
start, target = "damp", "like"
print(word_transform_dfs(start, target, dictionary))

['damp', 'lamp', 'limp', 'lime', 'like']
